In [2]:
from scipy.stats import norm
import numpy as np
import yfinance as yf
import pandas as pd

from datetime import datetime


class BsOption:
    def __init__(self, S, K, T, r, sigma, q=0):
        self.S = S
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.q = q

    @staticmethod
    def N(x):
        return norm.cdf(x)

    @property
    def params(self):
        return {'S': self.S,
                'K': self.K,
                'T': self.T,
                'r': self.r,
                'q': self.q,
                'sigma': self.sigma}

    def d1(self):
        return (np.log(self.S / self.K) + (self.r - self.q + self.sigma ** 2 / 2) * self.T) \
               / (self.sigma * np.sqrt(self.T))

    def d2(self):
        return self.d1() - self.sigma * np.sqrt(self.T)

    def _call_value(self):
        return self.S * np.exp(-self.q * self.T) * self.N(self.d1()) - \
               self.K * np.exp(-self.r * self.T) * self.N(self.d2())

    def _put_value(self):
        return self.K * np.exp(-self.r * self.T) * self.N(-self.d2()) - \
               self.S * np.exp(-self.q * self.T) * self.N(-self.d1())

    def price(self):
        # return self._call_value()
        return self._call_value(), self._put_value()


def GetData(stk, start_date, end_date):
    df = yf.download(stk, start_date, end_date, interval='1d').dropna()
    return (df)


def AddTime(df):
    df['start']=df.index
    df['end']=df.index
    df['dow'] = df.index.dayofweek
    #df['moy']
    df['woy'] = df.index.isocalendar().week + df.index.isocalendar().year * 100
    df['moy']=df.index.strftime('%m-%Y')
    return (df)

def GroupW(df,div,type):
    #df=df[df['dow']!=3]
    #df=df[df['dow']!=2]
    #df=df[df['dow']!=1]
    #df=df[df['dow']!=0]
    if type=="week":
        df['woy']=(df.woy/div).apply(np.ceil)
        df = df.groupby(['woy']).agg({'start':'first','end':'last','Open': 'first', 'Close': 'last'})
        df['days']= (df['end'] - df['start']).dt.days+1
        df=df.set_index('start')
    #if type=="month":
    #    df['moy'] = (df.moy/div).apply(np.ceil)
    #    df = df.groupby(['moy']).agg({'Open': 'first', 'Close': 'last'})
    return (df)

def Outs(df,IV):
    df['K'] = df.Open.round(0)
    df['C1'] = BsOption(df.Open, df.K, df.days / 365, 0.03, IV).price()[0]
    df['C2'] = BsOption(df.Close, df.K, 0 / 365, 0.03, IV).price()[0]
    df['P1'] = BsOption(df.Open, df.K, df.days / 365, 0.03, IV).price()[1]
    df['P2'] = BsOption(df.Close, df.K, 0 / 365, 0.03, IV).price()[1]
    df['T1'] = df.C1 + df.P1
    df['T2'] = df.C2 + df.P2
    df['PL'] = df.T1 - df.T2
    df['LogR'] = np.log((df.Open + df.PL) / df.Open)
    df['OptionYield']=df.T1/df.K
    return(df)

#Inputs
intervals=[1,2,4,12,24,48]
stock='XLC'
#start_date='2020-01-13'
#end_date='2020-12-24'

#start_date='2021-01-11'
#end_date='2021-12-24'

#start_date='2022-01-03'
#end_date='2022-06-25'

start_date='2020-01-01'
end_date='2022-08-06'

IV = 0.365
div=2
type="week"


#Execution
df = GetData(stock, start_date, end_date)
df = AddTime(df)
df=GroupW(df,div,type)
df=Outs(df,IV)


print(df)
filename=(stock+", "+start_date+", "+end_date+", "+str(div)+".csv")
df.to_csv(filename)
#Sum Log Returns
print(np.sum(df.LogR))
#Sum Positive Log
print(np.sum(df[df.LogR > 0].LogR))
#Sum Negative Log
print(np.sum(df[df.LogR <= 0].LogR))
#Count of Positive Periods
print(len(df[df.LogR > 0].LogR))
#Total Count of Periods
print(len(df.LogR))
#Average Option Yield
print(np.average(df.OptionYield))
print((np.sum(df[df.LogR > 0].LogR))/(-(np.sum(df[df.LogR <= 0].LogR))))
#Days/Period

[*********************100%***********************]  1 of 1 completed
                  end       Open      Close  days     K        C1        C2  \
start                                                                         
2020-01-02 2020-01-10  53.970001  55.160000     9  54.0  1.238732  1.160000   
2020-01-13 2020-01-24  55.419998  55.689999    12  55.0  1.706075  0.689999   
2020-01-27 2020-02-07  54.650002  55.860001    12  55.0  1.303461  0.860001   
2020-02-10 2020-02-21  55.730000  56.119999    12  56.0  1.369076  0.119999   
2020-02-24 2020-03-06  54.400002  49.820000    12  54.0  1.668084  0.000000   
...               ...        ...        ...   ...   ...       ...       ...   
2022-06-06 2022-06-17  60.279999  53.840000    12  60.0  1.762118  0.000000   
2022-06-21 2022-07-01  54.470001  54.820000    11  54.0  1.645131  0.820000   
2022-07-05 2022-07-15  54.130001  55.410000    11  54.0  1.457103  1.410000   
2022-07-18 2022-07-29  55.799999  56.369999    12  56.0  1.403